<div align=center>

# **_Principles of Simulation_**

## **Final Project**

By Hamed Araab & Shahriar Khalvati

</div>


### Prerequisites

In this section, we import necessary libraries and modules required for the
execution of subsequent code cells:


In [ ]:
import random
import pandas as pd
import seaborn as sns

from framework import *

### Servers

Doctors are servers. Two types of doctors are availables.


In [3]:
class Doctor:
    def __init__(self, controller) -> None:
        self.controller = controller
        self.status: Literal["available", "busy"] = "available"

class Examiner(Doctor):
    pass

class Physician(Doctor):
    pass

1

In [ ]:
class DoctorSelectionStrategy:
    @staticmethod
    def randomAvailable(doctors: List[Doctor]) -> Doctor:
        doctors = doctors.copy()
        selectedDoctor: Doctor | None = None

        random.shuffle(doctors)

        for doctor in doctors:
            if doctor.status == "available":
                selectedDoctor = doctor

                break

        return selectedDoctor

In [ ]:
class Patient:
    pass

class HighPriorityPatient(Patient):
    pass

class LowPriorityPatient(Patient):
    pass

In [ ]:
class HospitalController(SimController):
    def __init__(self, examinersCount=1, physiciansCount=1) -> None:
        super().__init__(stopTime=32 * 24 * 60, initialEvent=ArrivalEvent(initial=True))
        self.treatedPatients: List[Patient] = []
        self.highPriorityExaminationQueue: List[HighPriorityPatient] = []
        self.lowPriorityExaminationQueue: List[LowPriorityPatient] = []
        self.treatmentQueue: List[Patient] = []

        self.examiners: List[Examiner] = [
            Examiner(controller=self)
            for i in range(1, examinersCount + 1)
        ]

        self.physicians: List[Physician] = [
            Physician(controller=self)
            for i in range(1, physiciansCount + 1)
        ]

In [ ]:
ARRIVAL_EVENT_INTERVAL: Callable[[], float] | None = None


class ArrivalEvent(SimEvent[HospitalController]):
    def __init__(self, initial: bool = False) -> None:
        super().__init__(0 if initial else ARRIVAL_EVENT_INTERVAL())

        self.patient = (
            LowPriorityPatient()
            if random.random() > 0.15
            else HighPriorityPatient()
        )

    def trigger(self) -> None:
        self.controller.dispatchEvent(ArrivalEvent())

        appropriateQueue = (
            self.controller.highPriorityExaminationQueue
            if self.patient is HighPriorityPatient
            else self.controller.lowPriorityExaminationQueue
        )

        selectedExaminer = DoctorSelectionStrategy.randomAvailable(self.controller.examiners)

        if appropriateQueue or not selectedExaminer:
            appropriateQueue.append(self.patient)
        else:
            selectedExaminer.status = "busy"
                
            self.controller.dispatchEvent(
                ExaminationFinishEvent(
                    patient=self.patient,
                    examiner=selectedExaminer,
                )
            )

In [ ]:
EXAMINATION_FINISH_EVENT_INTERVAL: Callable[[], float] | None = None


class ExaminationFinishEvent(SimEvent[HospitalController]):
    def __init__(self, patient: Patient, examiner: Examiner) -> None:
        super().__init__()

        self.patient = patient
        self.examiner = examiner

    def trigger(self) -> None:
        selectedPhysician = DoctorSelectionStrategy.randomAvailable(self.controller.physicians)

        if self.controller.treatmentQueue or not selectedPhysician:
            self.controller.treatmentQueue.append(self.patient)
        else:
            selectedPhysician.status = "busy"

            self.controller.dispatchEvent(
                TreatmentFinishEvent(
                    patient=self.patient,
                    physician=selectedPhysician,
                )
            )

        if self.controller.highPriorityExaminationQueue:
            patient = self.controller.highPriorityExaminationQueue.pop(0)
        elif self.controller.lowPriorityExaminationQueue:
            patient = self.controller.lowPriorityExaminationQueue.pop(0)
        else:
            patient = None

        if patient:
            self.controller.dispatchEvent(
                ExaminationFinishEvent(
                    patient=patient,
                    examiner=self.examiner,
                )
            )
        else:
            self.examiner.status = "available"

In [ ]:
TREATMENT_FINISH_EVENT_INTERVAL: Callable[[], float] | None = None


class TreatmentFinishEvent(SimEvent[HospitalController]):
    def __init__(self, patient: Patient, physician: Physician) -> None:
        super().__init__()

        self.patient = patient
        self.physician = physician

    def trigger(self) -> None:
        self.controller.treatedPatients.append(self.patient)

        patient = (
            self.controller.treatmentQueue.pop(0)
            if self.controller.treatmentQueue
            else None
        )

        if patient:
            self.controller.dispatchEvent(
                TreatmentFinishEvent(
                    patient=patient,
                    physician=self.physician,
                )
            )
        else:
            self.physician.status = "available"